In [1]:
import pandas as pd
from ast import literal_eval

metadata = pd.read_csv('./data/michgov6122023.csv').dropna(subset='Body').reset_index(drop=True)
for col in ['To', 'Cc']:
    metadata[col] = metadata[col].apply(literal_eval)
metadata

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,1,0,From: Julie Skubik <julie@mhealthfund.com> Sen...,Staff_4_0000.txt,Julie Skubik <julie@mhealthfund.com,"Wednesday, June 03, 2015 3.34 PM","[;, Wendy (GOV)]",[],RE: Flint Water Situation MHEF Meeting,NaN,\nThank you. Do you need meto send an invite?\n
1,2,0,From: Wisniewski; Wendy (GOV) [mailto:wisniews...,Staff_4_0000.txt,Wisniewski; Wendy (GOV),"Wednesday, June 3,2015 3.33 PM",[Julie Skubik],[],RE: Flint Water Situation MHEF Meeting,NaN,\nAbsclutely. Here is all of my contact inforr...
2,3,0,From: Julie Skubik [mailtotiule@mhealthfundcom...,Staff_4_0000.txt,Julie Skubik,"Wednesday, June 03,2015 3.30 PM","[Wisniewski, Wendy (GOV)]",[],FW: Flint Water Situation MHEF Meeting,NaN,"\nHi Wendy- June 10""* at 11:0O am w} Paul Hill..."
3,4,0,"From: Hovey, Amy [mailto Amx Hovex@mailhousego...",Staff_4_0000.txt,"Hovey, Amy","Wednesday, June 3,2015 1.56 PM",[Julie Skubik],[],RE: Flint Water Situation MHEF Meeting,NaN,"\nHi Julie\nJune 10t at 11AMwculd be great, Wh..."
4,5,0,From: Julie Skubik [mailto iulie@mhezlthfundco...,Staff_4_0000.txt,Julie Skubik,"Wednesday, June 03, 2015 10.59 AM","[Hovey, Amy Subject:]",[],Flint Water Situation MHEF Meeting,NaN,"\nXi Arny"" would Ilke to set up & meeting with..."
...,...,...,...,...,...,...,...,...,...,...,...
26154,32176,16920,"From: Stoken, Laura (GOV) Sent: MondayDecember...",Staff_2_0920.txt,"Stoken, Laura (GOV)",NaN,[],[],Executive Office of the Governor\n,NaN,Dear Thank you for your recent correspondence ...
26155,32178,16921,"From: Stoken, Laura (GOV) Sent: Mondav_Decembe...",Staff_2_0922.txt,"Stoken, Laura (GOV)",NaN,[],[],Executive Office of the Governor\n,NaN,Dear Thank you for your recent correspondence ...
26156,32180,16922,From: Stoken; Laura (GOV) Sent: Monday Decembe...,Staff_2_0924.txt,Stoken; Laura (GOV),"Monday December 28, 2015 10.50 AM",[],[],Executive Office of the Governor\nDearl,NaN,Thank you for your recent correspondence sent...
26157,32182,16923,"From: Stoken, Laura (GOV) Sent: MondayDecember...",Staff_2_0926.txt,"Stoken, Laura (GOV)",NaN,[],[],Executive Office of the Governor\n,NaN,Dear Thank you for your recent correspondence ...


In [12]:
import spacy
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")

docs = metadata['Body'].apply(nlp)

In [15]:
sents = docs.apply(lambda x: [s.text for s in x.sents]).explode()
sents = sents.reset_index().rename(columns={
    'index':'org_index'
})
sents
# sents = [item for sublist in docs for item in sublist]
# sents[0]

,org_index,Body
0,0,\nThank you.
1,0,Do you need meto send an invite?
2,0,\n
3,1,\nAbsclutely.
4,1,Here is all of my contact inforration:\nWendy ...
...,...,...
111622,26158,"During the same months in 2014, the figure inc..."
111623,26158,\nOn October 21 Governor Snyder announced the ...
111624,26158,\nMDHHS continues to provide free water filter...
111625,26158,For a full list of locations and hours of dist...


In [47]:
sents.to_csv('./data/fl_sents6132023.csv')

In [4]:
from txtai.embeddings import Embeddings
embeddings = Embeddings()
embeddings.load('./data/fl_index6132023')

In [16]:
embeddings.search('query: contaminated water', limit=5)

[(49022, 0.8616752624511719),
 (18164, 0.8591524958610535),
 (17503, 0.8591524958610535),
 (538, 0.8591524958610535),
 (82397, 0.854966938495636)]

In [18]:
sents.iloc[82397].Body

'The corrosive attributes of the River water have exacerbated the lead deposit leaching into homeowner pipes and there is concern around lead poisoning; especially for infants.'

In [20]:
metadata.iloc[sents.iloc[82397].org_index].Body

' <baitd@qichigegexsmailtobeird @ichigangew> > wrote:\nFrank:\nWe have a situation in Flint, Michigan that is beginning to get national attention and could be an opportunity for Home Depot 18 months ago, Flint switched its water supply from Detroit Water (the distance and pumping costs of going uphill made it quite expensive) to the Flint River as a temporary answer; until they could finish the construction to get their water from Lake Huron (Karagondi Water System) . The corrosive attributes of the River water have exacerbated the lead deposit leaching into homeowner pipes and there is concern around lead poisoning; especially for infants.\nThe corporate community is rallying to address the problem: Meijer donated 1500 water filters to low income residents and GM is about to announce a donation of between 5,000 and 10,000 filters Ido not yet know where GM is getting the filters from; but I was thinking that Home Depot might want to join the effort with a donation, supply chain discoun

In [24]:
metadata.iloc[sents.iloc[82397].org_index].image_lookup

'Staff_10_0431.txt'

In [49]:
from thefuzz import fuzz

to_see = 10
uids = embeddings.search('query: contaminated water', limit=to_see+100) # arbitrary
strings = [(sents.iloc[uid[0]].Body, uid[0], uid[1]) for uid in uids]
unique_strings = []

for string in strings:
    is_dup = False
    for unique in unique_strings:
        fuzz_score = fuzz.ratio(string[0], unique[0])
        if fuzz_score >= 90:
            is_dup = True
            break
    if not is_dup:
        unique_strings.append(string)

In [50]:
unique_strings[:10]

[('\nhttp Iflintwaterstudy Q.g/20LS/L2Inichigan-healh-departnent-bid-evidence-efhealth-bar-dueto-lead contaminated-water-allewed-false-public-assurances-by-mdeg-and-stonewalled-eutside-Iesearchers?',
  49022,
  0.8616752624511719),
 ("\nwater; and the city's failure to adequately treat it. #",
  18164,
  0.8591524958610535),
 ('The corrosive attributes of the River water have exacerbated the lead deposit leaching into homeowner pipes and there is concern around lead poisoning; especially for infants.',
  82397,
  0.854966938495636),
 ('The people of Flint; Michigan, have been poisoned as a result of the decision to switched from a safe water source to a contaminated water source.',
  15527,
  0.8537269830703735),
 ('An occurrence of a waterborne disease outbreak; 4 A failure or significant interruption in key water treatment processes; 5.',
  41467,
  0.8537167310714722),
 ('Contaminants thatmay be presert in water include micrchial contaminants Such &s viruses and bacterja, and inorga

In [52]:
new_uids = [(u[1], u[2]) for u in unique_strings][:10]
new_uids

[(49022, 0.8616752624511719),
 (18164, 0.8591524958610535),
 (82397, 0.854966938495636),
 (15527, 0.8537269830703735),
 (41467, 0.8537167310714722),
 (68823, 0.8532322645187378),
 (49066, 0.853163480758667),
 (84881, 0.8522719740867615),
 (17187, 0.8514165878295898),
 (38983, 0.8507027626037598)]

In [26]:
dit_logits = pd.read_csv('./data/dit_logits_embedded.csv')
dit_logits.aws_path.loc[dit_logits.aws_path.str.contains('Staff_10_0431')].iloc[0]

'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_10_jp2/Staff_10_0431.png'

In [44]:
from sentence_transformers import CrossEncoder
def retrieval_rerank(query, model_path, rerank_model_path, input_texts, limit):
    embeddings = Embeddings()
    embeddings.load(model_path)
    s = embeddings.search(query, limit=limit)
    rerank_model = CrossEncoder(rerank_model_path, max_length=512)
    scores = rerank_model.predict([(query, input_texts[tup[0]]) for tup in s])
    reranked = [s[score][0] for score in scores.argsort()[::-1]]
    return reranked

In [45]:
reranked = retrieval_rerank(
    'neglience of government officials',
    './data/fl_index622023',
    'cross-encoder/ms-marco-MiniLM-L-2-v2',
    input_texts,
    10
)
reranked

[11802, 4595, 17313, 16927, 4863, 6970, 20752, 5833, 16166, 7922]

In [48]:
def lookup_image(rerank_id):
    return dit_logits.aws_path.loc[dit_logits.aws_path.str.contains(metadata.iloc[rerank_id].image_lookup.split('.')[0])].iloc[0]

In [49]:
[lookup_image(rr) for rr in reranked]

['https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_5_jp2/Staff_5_0478.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_18_jp2/Staff_18_0031.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_17_jp2/Staff_17_0321.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_17_jp2/Staff_17_0046.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_18_jp2/Staff_18_0230.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_12_jp2/Staff_12_0323.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_15_jp2/Staff_15_0097.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_13_jp2/Staff_13_0708.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_14_jp2/Staff_14_1688.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_12_jp2/Staff_12_1171.png']

In [64]:
thread_id = metadata.iloc[2505].thread_index
metadata.iloc[metadata.groupby('thread_index').groups[thread_id]].image_lookup.unique()[0]

'Staff_9_0860.txt'